in here we will be building an alarm system/motion detection based off of these three videos. the plan is to make an alarm system that captures the frame where the motion was detected mostly all the frames containing motion detection

we are first going to import 
cv2 for webcam image handling
imutils for cropping resizing
threading to handle multiple frames
winsound for alarm system

In [13]:
import cv2
import imutils
import threading
import winsound

we then setup the webcamfeed variable, height and width

In [25]:
cap = cv2.VideoCapture(1)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)


True

we will get a frame and an immediate netxt frame. if there is enough differences between the frame we will sound the allarm
so, we define a start frame, set its width to 500, convert the frame wo grayscale to view changes, then we use a gausian blur on the image to check for deviation

In [26]:
ret, start_frame = cap.read()
start_frame = imutils.resize(start_frame, width=500)
start_frame = cv2.cvtColor(start_frame, cv2.COLOR_BGR2GRAY)
start_frame = cv2.GaussianBlur(start_frame, (21, 21), 0)

variable that check if an alarm is active, alarm mode  to check if alarm is toggled and alarm countr to set duration of alarm

In [27]:
alarm = False
alarm_mode = False
alarm_counter = 0


define a simple alarm function that defines what happens when an alarm is set off
windows.beep sets the frequency pitch of the sound
we want to produce 5 consecutive one second beeps

In [28]:
def beep_alarm():
    global alarm
    for _ in range(5):
        if not alarm_mode:
            break
        print("Alarm")
        winsound.Beep(2500, 1000)
    alarm = False




In [29]:
while True:
    ret, frame = cap.read()
    frame = imutils.resize(frame, width=500)

    if alarm_mode:
        frame_bw = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_bw = cv2.GaussianBlur(frame_bw, (5, 5), 0)

        difference = cv2.absdiff(frame_bw, start_frame)
        threshold = cv2.threshold(difference, 25, 255, cv2.THRESH_BINARY)[1]
        start_frame = frame_bw

        if threshold.sum() > 300:
            alarm_counter += 1
        else:
            if alarm_counter > 0:
                alarm_counter -= 1

        cv2.imshow("Cam", threshold)
    else:
        cv2.imshow("Cam", frame)

    if alarm_counter > 20 :
        if not alarm:
            alarm = True
            threading.Thread(target=beep_alarm).start()

    key_pressed = cv2.waitKey(30)
    if key_pressed == ord("t"):
        alarm_mode = not alarm_mode
        alarm_counter = 0
    if key_pressed == ord("q"):
        alarm_mode = False
        break

cap.release()
cv2.destroyAllWindows()
        

so , we are going to fetch the frame from webcam, resize the frame to 500

```python
while True:
    ret, frame = cap.read()
    frame = imutils.resize(frame, width=500)
```

if the alarm mode is activated then we will calculate differences btwn the frames
we will convert the current frame to black and white then pass it through a gausian blur to reduce noise and smoothen edges.this is because we want to detect movement in edges of the person in view

we then calculate the diff btwn the initial frame ie start frame and frame_bw
we then create a threshold such that everything above 25 is 1 and everything below 25 is 0 ie threshbinasry
then we set startframe as the frame_bw


then, if sum of threshold is above 300, set alarm counter plus 1 otherwise reduce alarm counter
else if there is no alarm mode, just show the frame
```python
    if alarm_mode:
        frame_bw = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_bw = cv2.GaussianBlur(frame_bw, (5, 5), 0)

        difference = cv2.absdiff(frame_bw, start_frame)
        threshold = cv2.threshold(difference, 25, 255, cv2.THRESH_BINARY)[1]
        start_frame = frame_bw

        if threshold.sum() > 300:
            alarm_counter += 1
        else:
            if alarm_counter > 0:
                alarm_counter -= 1

        cv2.imshow("Cam", threshold)
    else:
        cv2.imshow("Cam", frame)
```
if alarm counter exceeds 20, sound the alarm 
```python
    if alarm_counter > 20 :
        if not alarm:
            alarm = True
            threading.Thread(target=beep_alarm).start()
```
set keybord press functions
```python
    key_pressed = cv2.waitKey(30)
    if key_pressed == ord("t"):
        alarm_mode = not alarm_mode
        alarm_counter = 0
    if key_pressed == ord("q"):
        alarm_mode = False
        break
```

cap.release()
cv2.destroyAllWindows()

In [23]:
# play a sound on a Windows Box (Python2 or 3)
# Playsound(sound, flags) accepts wave sound files and uses
# the external speaker system
# Beep(frequency, duration) uses the internal speaker of the computer

import winsound as ws
import time

# pick a wave file supplied by Windows XP or one of your own ...
soundfile = "C:/Windows/Media/chimes.wav"
ws.PlaySound(soundfile, ws.SND_FILENAME|ws.SND_ASYNC)

# wait one and a half seconds
time.sleep(1.5)

# play the system exit sound if set
ws.PlaySound("SystemExit", ws.SND_ALIAS)

time.sleep(1.5)

frequency = 2500  # hertz or cps
duration = 1000  # milliseconds
ws.Beep(frequency, duration)

# help('winsound')